In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
import tensorflow
print(tensorflow.__version__)
import tensorflow.keras as keras

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, \
  concatenate, Dense, Activation, LSTM, Dropout, Bidirectional, Flatten

import numpy as np
from sklearn.model_selection import StratifiedKFold

2.1.0-rc1


In [0]:
# Parameters:
# Embedding
max_features = 20000
maxlen = 400
embedding_size = 128

# Convolution
kernel_size = 3
filters = 128
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
epochs = 5

In [4]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
x_data = np.concatenate((x_train, x_test))
y_data = np.concatenate((y_train, y_test))
skf = StratifiedKFold(n_splits=5, random_state=42)
print('x_data shape:', x_data.shape)
print('y_data shape:', y_data.shape)

Loading data...
17465344/17464789 [==============================] - 1s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)
x_data shape: (50000, 400)
y_data shape: (50000,)


In [5]:
# Network Architecture
def create_CNNLSTM_model():
  # CNN + LSTM 
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_size,
                      input_length=maxlen)(var_input)
  conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='valid',activation='relu',
                  strides=1)(emb_layer)
  pool_layer_1 = MaxPooling1D(pool_size=pool_size)(conv_layer_1)
  lstm_layer = LSTM(units=lstm_output_size, activation='tanh', recurrent_activation='sigmoid', 
                    dropout=0.0, recurrent_dropout=0.0)(pool_layer_1)
  dense_layer = Dense(1, activation='sigmoid')(lstm_layer)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
  return model

create_CNNLSTM_model().summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 400)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 400, 128)          2560000   
_________________________________________________________________
conv1d (Conv1D)              (None, 398, 128)          49280     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 99, 128)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 70)                55720     
_________________________________________________________________
dense (Dense)                (None, 1)                 71        
Total params: 2,665,071
Trainable params: 2,665,071
Non-trainable params: 0
___________________________________________________

In [6]:
for train, test in skf.split(x_data, y_data):
  print('train data shape:', train.shape)
  print('test data shape:', test.shape)
  target = create_CNNLSTM_model()
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 62s 2ms/sample - loss: 0.3287 - accuracy: 0.8551 - val_loss: 0.2548 - val_accuracy: 0.9004
Epoch 2/5
40000/40000 [==============================] - 54s 1ms/sample - loss: 0.1725 - accuracy: 0.9364 - val_loss: 0.2778 - val_accuracy: 0.8923
Epoch 3/5
40000/40000 [==============================] - 54s 1ms/sample - loss: 0.0972 - accuracy: 0.9680 - val_loss: 0.3549 - val_accuracy: 0.8879
Epoch 4/5
40000/40000 [==============================] - 54s 1ms/sample - loss: 0.0539 - accuracy: 0.9832 - val_loss: 0.3996 - val_accuracy: 0.8892
Epoch 5/5
40000/40000 [==============================] - 54s 1ms/sample - loss: 0.0341 - accuracy: 0.9891 - val_loss: 0.4093 - val_accuracy: 0.8747
train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================]

In [7]:
# Network Architecture
def create_CNNBLSTM_model():
  # CNN + Bidirectional LSTM
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_size,
                      input_length=maxlen)(var_input)
  conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='valid',activation='relu',
                  strides=1)(emb_layer)
  pool_layer_1 = MaxPooling1D(pool_size=pool_size)(conv_layer_1)
  lstm_layer = Bidirectional(LSTM(units=lstm_output_size, activation='tanh', recurrent_activation='sigmoid', 
                    dropout=0.0, recurrent_dropout=0.0))(pool_layer_1)
  dense_layer = Dense(1, activation='sigmoid')(lstm_layer)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
  return model

create_CNNBLSTM_model().summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 400)]             0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 400, 128)          2560000   
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 398, 128)          49280     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 99, 128)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 140)               111440    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 141       
Total params: 2,720,861
Trainable params: 2,720,861
Non-trainable params: 0
_________________________________________________

In [8]:
for train, test in skf.split(x_data, y_data):
  print('train data shape:', train.shape)
  print('test data shape:', test.shape)
  target = create_CNNBLSTM_model()
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 63s 2ms/sample - loss: 0.3281 - accuracy: 0.8544 - val_loss: 0.2547 - val_accuracy: 0.8998
Epoch 2/5
40000/40000 [==============================] - 60s 1ms/sample - loss: 0.1688 - accuracy: 0.9388 - val_loss: 0.2728 - val_accuracy: 0.8959
Epoch 3/5
40000/40000 [==============================] - 60s 1ms/sample - loss: 0.0974 - accuracy: 0.9663 - val_loss: 0.3329 - val_accuracy: 0.8869
Epoch 4/5
40000/40000 [==============================] - 60s 1ms/sample - loss: 0.0527 - accuracy: 0.9836 - val_loss: 0.4423 - val_accuracy: 0.8884
Epoch 5/5
40000/40000 [==============================] - 60s 1ms/sample - loss: 0.0344 - accuracy: 0.9891 - val_loss: 0.4249 - val_accuracy: 0.8886
train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================]

In [12]:
# Network Architecture
def create_BLSTMCNN_model():
  # Bidirectional LSTM + CNN
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_size,
                      input_length=maxlen)(var_input)
  lstm_layer = Bidirectional(LSTM(units=lstm_output_size, activation='tanh', recurrent_activation='sigmoid', 
                    dropout=0.0, recurrent_dropout=0.0, return_sequences=True))(emb_layer)
  conv_layer_1 = Conv1D(filters, kernel_size=kernel_size, padding='valid',activation='relu',
                  strides=1)(lstm_layer)
  pool_layer_1 = MaxPooling1D(pool_size=pool_size)(conv_layer_1)
  flattened = Flatten()(pool_layer_1)

  dense_layer = Dense(1, activation='sigmoid')(flattened)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])
  return model

create_BLSTMCNN_model().summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 400)]             0         
_________________________________________________________________
embedding_16 (Embedding)     (None, 400, 128)          2560000   
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 400, 140)          111440    
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 398, 128)          53888     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 99, 128)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 12672)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 126

In [13]:
for train, test in skf.split(x_data, y_data):
  target = create_BLSTMCNN_model()
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 102s 3ms/sample - loss: 0.2973 - accuracy: 0.8679 - val_loss: 0.2354 - val_accuracy: 0.9058
Epoch 2/5
40000/40000 [==============================] - 99s 2ms/sample - loss: 0.1420 - accuracy: 0.9468 - val_loss: 0.2604 - val_accuracy: 0.9035
Epoch 3/5
40000/40000 [==============================] - 101s 3ms/sample - loss: 0.0746 - accuracy: 0.9733 - val_loss: 0.3109 - val_accuracy: 0.9011
Epoch 4/5
40000/40000 [==============================] - 101s 3ms/sample - loss: 0.0353 - accuracy: 0.9879 - val_loss: 0.4964 - val_accuracy: 0.8989
Epoch 5/5
40000/40000 [==============================] - 101s 3ms/sample - loss: 0.0273 - accuracy: 0.9912 - val_loss: 0.5410 - val_accuracy: 0.8939
train data shape: (40000,)
test data shape: (10000,)
Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [===========================